<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [14]:
%%writefile Lotus.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/grocery"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_pages)

    def parse_pages(self, response):
        self.driver.get(response.url)

        try:
            WebDriverWait(self.driver, 10).until(
                expected_conditions.presence_of_element_located(
                    (By.CLASS_NAME, "product-grid-item")
            ))

            selector = scrapy.Selector(text=self.driver.page_source)

            for page in selector.css("div.product-grid-item"):
                pageItem = page.css("a::attr(href)").get()
                if pageItem is not None:
                    yield response.follow(pageItem,
                        callback=self.parse_item)
        finally:
            self.driver.quit()

    def parse_item(self, response):
        print(response.url)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [ ]:
%%bash
python Lotus.py